In [2]:
import pandas as pd
import numpy as np


tourney_seeds = pd.read_csv("./Data/TourneySeeds.csv")
rank = []
teams = []
for seed, team in tourney_seeds.loc[:,['Seed','Team']].values:
    rank.append(int(seed[1:3]))
tourney_seeds["NumericalSeeds"] = rank

tourney_compact_results = pd.read_csv("./Data/TourneyCompactResults.csv")

result = pd.DataFrame({
        "Season": pd.concat([tourney_compact_results['Season'], tourney_compact_results['Season']], ignore_index=True),
        "Player": pd.concat([tourney_compact_results['Wteam'], tourney_compact_results['Lteam']], ignore_index=True),
        "Opponent": pd.concat([tourney_compact_results['Lteam'], tourney_compact_results['Wteam']], ignore_index=True),
        "Result": np.hstack([np.ones(tourney_compact_results.shape[0], dtype=int), np.zeros(tourney_compact_results.shape[0], dtype=int)])
    }, columns=["Result", "Season", "Player", "Opponent"])

t = pd.merge(result, tourney_seeds, left_on=['Season', 'Player'], right_on=['Season', 'Team'])
t = t[['Result', 'Season','Player', 'Opponent', 'NumericalSeeds']]
t.columns=['Result', 'Season','Player', 'Opponent', 'PlayerSeed']
result = t

t = pd.merge(result, tourney_seeds, left_on=['Season', 'Opponent'], right_on=['Season', 'Team'])
t = t[['Result', 'Season', 'Player', 'Opponent', 'PlayerSeed', 'NumericalSeeds']]
t.columns = ['Result', 'Season','Player', 'Opponent', 'PlayerSeed', 'OpponentSeed']
result = t

In [3]:
regular = pd.read_csv("./Data/RegularSeasonDetailedResults.csv")
regular.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


# try to get OD rank

In [3]:
table = regular[regular['Season']==2013][['Wteam', 'Lteam', 'Wscore', 'Lscore']].copy()

In [4]:
table.columns = [['primary', 'secondary', 'rate1', 'rate2']]
table['weight'] = np.ones(table.shape[0])

In [8]:
table.head()

,primary,secondary,rate1,rate2,weight
49836,1104,1355,70,67,1.0
49837,1107,1182,69,66,1.0
49838,1114,1404,84,68,1.0
49839,1116,1358,73,68,1.0
49840,1120,1236,61,50,1.0


In [3]:
from rankit.manager import RankMerger
from rankit.util import Converter
from rankit.ranker import ColleyRank, MasseyRank, ODRank, MarkovRank, KeenerRank

In [10]:
cvt = Converter(table=table)

M=cvt.RateVoteMatrix()
ranker=ODRank(itemlist=cvt.ItemList())
x=ranker.rate(M)
r3=ranker.rank(x)

/home/ike/Projects/ranking/rankit/util/converter.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  pair = np.fromfunction(np.vectorize(lambda i, j: idx[temptable[i,j]]),


In [17]:
odrank = pd.DataFrame({'Team': np.require(r3.ix[:, 0].values, dtype=np.int), 'rate':r3.ix[:, 1].values}, columns=['Team', 'rate'])

In [18]:
odrank.head()

,Team,rate
0,1196,3042.331787
1,1231,3005.559326
2,1257,2978.741943
3,1181,2832.316895
4,1326,2827.627197


In [6]:
result['SeedDelta'] = (result['OpponentSeed']-result['PlayerSeed'])/15.0
result['SeedState'] = (17.0-result['PlayerSeed'].values)/16
result.head()

,Result,Season,Player,Opponent,PlayerSeed,OpponentSeed,SeedDelta,SeedState
0,1,1985,1116,1234,9,8,-0.066667,0.5000
1,0,1985,1116,1385,9,1,-0.533333,0.5000
2,1,1985,1207,1385,1,1,0.000000,1.0000
3,0,1985,1246,1385,12,1,-0.733333,0.3125
4,0,1985,1301,1385,3,1,-0.133333,0.8750


In [7]:
team = []
rating = []
annee = []
for year in range(2003, 2017):
    table = regular[regular['Season']==year][['Wteam', 'Lteam', 'Wscore', 'Lscore']].copy()
    table.columns = [['primary', 'secondary', 'rate1', 'rate2']]
    table['weight'] = np.ones(table.shape[0])
    cvt = Converter(table=table)
    M = cvt.RateVoteMatrix()
    ranker = ODRank(itemlist=cvt.ItemList())
    x = ranker.rate(M)
    r = ranker.rank(x)
    team.append(np.require(r.ix[:, 0].values, dtype=np.int))
    rating.append(r.ix[:, 1].values)
    annee.append(year*np.ones(r.shape[0], dtype=np.int))
    

/home/ike/Projects/ranking/rankit/util/converter.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  pair = np.fromfunction(np.vectorize(lambda i, j: idx[temptable[i,j]]),


In [8]:
odrank = pd.DataFrame({
        "Season": np.hstack(annee),
        "Team": np.hstack(team),
        "ODRating": np.hstack(rating)
    }, columns=["Season", "Team", "ODRating"])

In [22]:
odrank.head()

,Season,Team,ODRating
0,2003,1242,2809.920166
1,2003,1246,2732.404785
2,2003,1112,2712.529785
3,2003,1338,2703.432373
4,2003,1228,2677.451660


In [9]:
t = pd.merge(result, odrank, left_on=["Season", "Player"], right_on=["Season", "Team"])
t = pd.merge(t, odrank, left_on=["Season", "Opponent"], right_on=["Season", "Team"], suffixes=('Ply', 'Opp'))
result = t.drop(['TeamPly', 'TeamOpp'], axis=1)

In [10]:
result['ODDelta']=result['ODRatingPly']-result['ODRatingOpp']
from scipy import stats

cdf = dict()
hist, bin_edges = np.histogram(result['ODDelta'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['ODDelta'] = stats.rv_discrete(name='ODDelta', values=(bin_edges[:-1], hist))
result['ODDelta'] = cdf['ODDelta'].cdf(result['ODDelta'].values)*2-1.0

In [11]:
result.drop(['ODRatingPly', 'ODRatingOpp'], axis=1, inplace=True)
result.head()

,Result,Season,Player,Opponent,PlayerSeed,OpponentSeed,SeedDelta,SeedState,ODDelta
0,1,2003,1421,1411,16,16,0.000000,0.0625,-0.230853
1,0,2003,1421,1400,16,1,-1.000000,0.0625,-0.975930
2,0,2003,1163,1400,5,1,-0.266667,0.7500,-0.409190
3,0,2003,1277,1400,7,1,-0.400000,0.6250,-0.286652
4,0,2003,1345,1400,9,1,-0.533333,0.5000,-0.303063


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

seg = result[(result['Season'])<2013].copy()
X = seg.ix[:, [6,8]].values
Y = seg.ix[:, 0].values
param = {'C':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
lr = LogisticRegression()
cver = GridSearchCV(lr, param, scoring='log_loss', cv=3, n_jobs=-1)
cver.fit(X, Y)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [30]:
cver.best_params_

{'C': 0.5}

In [31]:
lr = LogisticRegression(C=0.5)
lr.fit(X, Y)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
test = result[(result['Season']==2013)].copy()
Xt = test.ix[:, [6,8]].values
Yt = test.ix[:, 0].values
Yp = lr.predict_proba(Xt)
test['Pred']=Yp[:, 1]

In [33]:
from sklearn.metrics import log_loss

log_loss(Yt, Yp)

0.60507560890185341

In [34]:
#keeps the best auc
from sklearn.metrics import roc_auc_score

roc_auc_score(Yt, Yp[:,1])

0.74515482290042334

# Submit result

In [35]:
submission = pd.read_csv("./Data/sample_submission.csv")
content = []
for row in submission.iterrows():
    content.append(map(int, row[1].ix[0].split('_')))

In [36]:
submission = pd.DataFrame(content, columns=['Season', 'Player', 'Opponent'])

In [37]:
# take features from existing data
t = pd.merge(submission, tourney_seeds, left_on=['Season', 'Player'], right_on=['Season', 'Team'])
t = t.ix[:, [0,1,2,5]]
t.columns = ['Season', 'Player', 'Opponent', 'PlayerSeed']
t = pd.merge(t, tourney_seeds, left_on=['Season', 'Opponent'], right_on=['Season', 'Team'])
t = t.ix[:, [0, 1, 2, 3, 6]]
t.columns = ['Season', 'Player', 'Opponent', 'PlayerSeed', 'OpponentSeed']
t['SeedDelta'] = (t['OpponentSeed']-t['PlayerSeed']+16.0)/31
t['SeedState'] = (17.0-t['PlayerSeed'].values)/16
t = pd.merge(t, feature, on=['Season', 'Player'])
t = pd.merge(t, feature, left_on=['Season', 'Opponent'], right_on=['Season', 'Player'], suffixes=["Ply", "Opp"])
t.drop('PlayerOpp', 1, inplace=True)
t = t.rename(columns={"PlayerPly": "Player"})

NameError: name 'feature' is not defined